<a href="https://colab.research.google.com/github/SachinBiswas7/-Bangla-Text-Violence-Nature-and-Target-Classification/blob/main/task_2_transformer.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from sklearn.model_selection import train_test_split

In [ ]:
#import train, test , validation file
train = pd.read_csv('train_modified.csv')
test = pd.read_csv('test_modified.csv')
val = pd.read_csv('dev_modified.csv')

In [ ]:
train.head()

,text,label,Target
0,যে দেশে সন্ত্রাসরা দেশ চালায়সে দেশে শান্তি কিভ...,1,TO
1,এই বিচার শেষ বিচার নয়।আসল বিচার হবে আল্লাহর আদ...,0,TO
2,আরব দেশগুলোকে বলব ভারতের সাথে সব ব্যবসা বাণিজ্...,2,TC
3,দেশটা সুস্থ নাই,0,TI
4,আপনার কথা দুঃখ জনক আগে বিডিও থাকলে কেন ধরা হলন...,0,TC


In [ ]:
#label encoder
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
train['Target'] = le.fit_transform(train['Target'])
val['Target'] = le.fit_transform(val['Target'])
test['Target'] = le.fit_transform(test['Target'])

In [ ]:
train.head()

,text,label,Target
0,যে দেশে সন্ত্রাসরা দেশ চালায়সে দেশে শান্তি কিভ...,1,2
1,এই বিচার শেষ বিচার নয়।আসল বিচার হবে আল্লাহর আদ...,0,2
2,আরব দেশগুলোকে বলব ভারতের সাথে সব ব্যবসা বাণিজ্...,2,0
3,দেশটা সুস্থ নাই,0,1
4,আপনার কথা দুঃখ জনক আগে বিডিও থাকলে কেন ধরা হলন...,0,0


In [ ]:
test.head()

,text,label,Target
0,বাংলাদেশের হিন্দুরা নিজেদের জন্য আলাদা হিন্দু ...,0.0,0
1,মাইজদী - চৌমুহুনী - ফেনী মন্দিরে হামলা নিয়ে রি...,0.0,1
2,"দয়া করে পবিত্র কুরআনুল কারিম বলেন,,,,পবিত্র কথ...",0.0,1
3,বিবিসি হলো সত্য কে বিনষ্টকারী আর মিথ্যা কে গ্র...,1.0,2
4,বুধবার কি তোরা মারা গেছিলি বিবিসি বাংলা,1.0,2


In [ ]:
#drop Racial biases
train.drop(['label'], axis=1, inplace=True)

In [ ]:
test.drop(['label'], axis=1, inplace=True)


In [ ]:
val.drop(['label'], axis=1, inplace=True)

In [ ]:
#data preprocessing using spacy

#removing unnecessary punctuation
import re

# Data cleaning function
def process_comments(Comment):
    Comment = re.sub('[^\u0980-\u09FF]',' ',str(Comment)) #removing unnecessary punctuation
    return Comment

train['Cleaned'] = train['text'].apply(process_comments)
train.head()

test['Cleaned'] = test['text'].apply(process_comments)
test.head()

val['Cleaned'] = val['text'].apply(process_comments)
val.head()

,text,Target,Cleaned
0,পাডা পুতার মাঝখানে পরে সাধারণ ২ মানুষের জিবন শ...,2,পাডা পুতার মাঝখানে পরে সাধারণ ২ মানুষের জিবন শ...
1,করোনার চাপে অনেক কিছু বন্ধ ও অনেক বিধি নিষেধ ক...,2,করোনার চাপে অনেক কিছু বন্ধ ও অনেক বিধি নিষেধ ক...
2,সঠিক তদন্ত করতে হবে। বিচারের আওতায় আনতে হবে য...,1,সঠিক তদন্ত করতে হবে বিচারের আওতায় আনতে হবে য...
3,যে লোকটা মারা গেছে তার কি হবে তার দায়ভার কে ন...,1,যে লোকটা মারা গেছে তার কি হবে তার দায়ভার কে ন...
4,নিউ মার্কেট এবং গুলিস্থান মার্কেটের ব্যবসায়ীর...,2,নিউ মার্কেট এবং গুলিস্থান মার্কেটের ব্যবসায়ীর...


In [ ]:
val.head()

,text,Target,Cleaned
0,পাডা পুতার মাঝখানে পরে সাধারণ ২ মানুষের জিবন শ...,2,পাডা পুতার মাঝখানে পরে সাধারণ ২ মানুষের জিবন শ...
1,করোনার চাপে অনেক কিছু বন্ধ ও অনেক বিধি নিষেধ ক...,2,করোনার চাপে অনেক কিছু বন্ধ ও অনেক বিধি নিষেধ ক...
2,সঠিক তদন্ত করতে হবে। বিচারের আওতায় আনতে হবে য...,1,সঠিক তদন্ত করতে হবে বিচারের আওতায় আনতে হবে য...
3,যে লোকটা মারা গেছে তার কি হবে তার দায়ভার কে ন...,1,যে লোকটা মারা গেছে তার কি হবে তার দায়ভার কে ন...
4,নিউ মার্কেট এবং গুলিস্থান মার্কেটের ব্যবসায়ীর...,2,নিউ মার্কেট এবং গুলিস্থান মার্কেটের ব্যবসায়ীর...


In [ ]:
import pandas as pd

# Load Bengali stop words from a CSV file
stop_words_df = pd.read_csv('stopwords_bangla.csv')  # Assuming the CSV has one column with stop words
stop_words_df=stop_words_df.drop(['Unnamed: 0'], axis=1)
# Convert the stop words to a list
stop_words_list = stop_words_df.iloc[:, 0].tolist()  # Convert the column to a list

# Check the stop words list
print("Stop words list:", stop_words_list[:10])  # Print first 10 stop words for verification


# Function to remove stop words
def remove_stop_words(text, stop_words):
    if isinstance(text, str):  # Ensure the text is a string
        # Tokenize text (simple split for Bengali)
        words = text.split()
        # Filter out stop words
        filtered_words = [word for word in words if word not in stop_words]
        # Join words back into a single string
        return ' '.join(filtered_words)
    else:
        return text  # If text is not a string, return it as is

# Assuming 'df' is the DataFrame and 'Cleaned' is the column with text data
# Check a few entries in the original DataFrame




Stop words list: ['অই', 'অগত্যা', 'অত: পর', 'অতএব', 'অথচ', 'অথবা', 'অধিক', 'অধীনে', 'অধ্যায়', 'অনুগ্রহ']


In [ ]:
# Apply the function to the DataFrame
train['filtered_text'] = train['Cleaned'].apply(lambda x: remove_stop_words(x, stop_words_list))


train.head()

,text,Target,Cleaned,filtered_text
0,যে দেশে সন্ত্রাসরা দেশ চালায়সে দেশে শান্তি কিভ...,2,যে দেশে সন্ত্রাসরা দেশ চালায়সে দেশে শান্তি কিভ...,দেশে সন্ত্রাসরা দেশ চালায়সে দেশে শান্তি আশা
1,এই বিচার শেষ বিচার নয়।আসল বিচার হবে আল্লাহর আদ...,2,এই বিচার শেষ বিচার নয় আসল বিচার হবে আল্লাহর আদ...,বিচার বিচার আসল বিচার আল্লাহর আদালতে সেইদিন সক...
2,আরব দেশগুলোকে বলব ভারতের সাথে সব ব্যবসা বাণিজ্...,0,আরব দেশগুলোকে বলব ভারতের সাথে সব ব্যবসা বাণিজ্...,আরব দেশগুলোকে বলব ভারতের সাথে ব্যবসা বাণিজ্য হ...
3,দেশটা সুস্থ নাই,1,দেশটা সুস্থ নাই,দেশটা সুস্থ
4,আপনার কথা দুঃখ জনক আগে বিডিও থাকলে কেন ধরা হলন...,0,আপনার কথা দুঃখ জনক আগে বিডিও থাকলে কেন ধরা হলন...,কথা দুঃখ জনক বিডিও থাকলে হলনা হিন্দু দের খতি হ...


In [ ]:
# Apply the function to the DataFrame
test['filtered_text'] = test['Cleaned'].apply(lambda x: remove_stop_words(x, stop_words_list))


test.head()

,text,Target,Cleaned,filtered_text
0,বাংলাদেশের হিন্দুরা নিজেদের জন্য আলাদা হিন্দু ...,0,বাংলাদেশের হিন্দুরা নিজেদের জন্য আলাদা হিন্দু ...,বাংলাদেশের হিন্দুরা আলাদা হিন্দু রাজ্য র দাবি ...
1,মাইজদী - চৌমুহুনী - ফেনী মন্দিরে হামলা নিয়ে রি...,1,মাইজদী চৌমুহুনী ফেনী মন্দিরে হামলা নিয়ে রি...,মাইজদী চৌমুহুনী ফেনী মন্দিরে হামলা রিপোর্ট করুন
2,"দয়া করে পবিত্র কুরআনুল কারিম বলেন,,,,পবিত্র কথ...",1,দয়া করে পবিত্র কুরআনুল কারিম বলেন পবিত্র কথ...,দয়া পবিত্র কুরআনুল কারিম পবিত্র কথাটা বলবেন প্লিজ
3,বিবিসি হলো সত্য কে বিনষ্টকারী আর মিথ্যা কে গ্র...,2,বিবিসি হলো সত্য কে বিনষ্টকারী আর মিথ্যা কে গ্র...,বিবিসি বিনষ্টকারী মিথ্যা গ্রহণকারী
4,বুধবার কি তোরা মারা গেছিলি বিবিসি বাংলা,2,বুধবার কি তোরা মারা গেছিলি বিবিসি বাংলা,বুধবার তোরা মারা গেছিলি বিবিসি বাংলা


In [ ]:
# Apply the function to the DataFrame
val['filtered_text'] = val['Cleaned'].apply(lambda x: remove_stop_words(x, stop_words_list))


val.head()


,text,Target,Cleaned,filtered_text
0,পাডা পুতার মাঝখানে পরে সাধারণ ২ মানুষের জিবন শ...,2,পাডা পুতার মাঝখানে পরে সাধারণ ২ মানুষের জিবন শ...,পাডা পুতার মাঝখানে ২ মানুষের জিবন বিচার না
1,করোনার চাপে অনেক কিছু বন্ধ ও অনেক বিধি নিষেধ ক...,2,করোনার চাপে অনেক কিছু বন্ধ ও অনেক বিধি নিষেধ ক...,করোনার চাপে বিধি নিষেধ নির্বাচন চলছেই কারন
2,সঠিক তদন্ত করতে হবে। বিচারের আওতায় আনতে হবে য...,1,সঠিক তদন্ত করতে হবে বিচারের আওতায় আনতে হবে য...,সঠিক তদন্ত বিচারের আওতায় আনতে
3,যে লোকটা মারা গেছে তার কি হবে তার দায়ভার কে ন...,1,যে লোকটা মারা গেছে তার কি হবে তার দায়ভার কে ন...,লোকটা মারা দায়ভার নিবে ছাত্র ব্যবসায়ী
4,নিউ মার্কেট এবং গুলিস্থান মার্কেটের ব্যবসায়ীর...,2,নিউ মার্কেট এবং গুলিস্থান মার্কেটের ব্যবসায়ীর...,নিউ মার্কেট গুলিস্থান মার্কেটের ব্যবসায়ীরা কু...


In [ ]:
!pip install torch transformers pandas numpy sklearn tqdm


  error: subprocess-exited-with-error
  
  × python setup.py egg_info did not run successfully.
  │ exit code: 1
  ╰─> See above for output.
  
  note: This error originates from a subprocess, and is likely not a problem with pip.
  Preparing metadata (setup.py) ... error
error: metadata-generation-failed

× Encountered error while generating package metadata.
╰─> See above for output.

note: This is an issue with the package mentioned above, not pip.
hint: See above for details.


In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
import pandas as pd
import numpy as np
import re
from torch.utils.data import Dataset, DataLoader
from transformers import AutoTokenizer, AutoModel
from tqdm import tqdm
from sklearn.metrics import accuracy_score, classification_report
from imblearn.over_sampling import RandomOverSampler  # Import RandomOverSampler
import unicodedata

# Load BanglaBERT Tokenizer
tokenizer = AutoTokenizer.from_pretrained("csebuetnlp/banglabert")

# Load Dataset
def load_data(file_path):
    df = pd.read_csv(file_path)
    df = df[['text', 'label']]
    return df

train_df = pd.read_csv("train_modified.csv")
dev_df = pd.read_csv("dev_modified.csv")
test_df = pd.read_csv("test_modified.csv")

# Step 1: Apply Random Oversampling to Balance the Datasets
def oversample_dataset(df):
    # Drop rows where 'text' or 'label' is NaN
    df = df.dropna(subset=['text', 'label'])

    # Ensure labels are integers
    df["label"] = df["label"].astype(int)

    ros = RandomOverSampler(random_state=42)
    texts, labels = df["text"].values.reshape(-1, 1), df["label"].values

    texts_resampled, labels_resampled = ros.fit_resample(texts, labels)

    balanced_df = pd.DataFrame({"text": texts_resampled.flatten(), "label": labels_resampled})
    return balanced_df

# Apply oversampling after fixing NaN issues
train_df = oversample_dataset(train_df)
dev_df = oversample_dataset(dev_df)
test_df = oversample_dataset(test_df)



# **Step 2: Preprocessing Function**
def preprocess_text(text):
    text = unicodedata.normalize('NFKC', text)  # Normalize Unicode text
    text = re.sub(r'[০-৯]', '[NUMBER]', text)  # Replace Bangla numbers
    text = re.sub(r'http\S+|www\S+', '[URL]', text)  # Replace URLs
    text = re.sub(r'[^\u0980-\u09FF\s]', '', text)  # Remove non-Bangla characters
    text = re.sub(r'\s+', ' ', text).strip()  # Normalize whitespace
    return text

train_df["text"] = train_df["text"].apply(preprocess_text)
dev_df["text"] = dev_df["text"].apply(preprocess_text)
test_df["text"] = test_df["text"].apply(preprocess_text)

# **Step 3: Define Dataset Class**
class TextDataset(Dataset):
    def __init__(self, texts, labels, tokenizer, max_length=70):
        self.texts = texts
        self.labels = labels
        self.tokenizer = tokenizer
        self.max_length = max_length

    def __len__(self):
        return len(self.texts)

    def __getitem__(self, idx):
        text = self.texts[idx]
        label = self.labels[idx]

        encoding = self.tokenizer(
            text,
            padding="max_length",
            truncation=True,
            max_length=self.max_length,
            return_tensors="pt"
        )

        return {
            "input_ids": encoding["input_ids"].squeeze(0),
            "attention_mask": encoding["attention_mask"].squeeze(0),
            "label": torch.tensor(label, dtype=torch.long)
        }

# Prepare datasets
train_dataset = TextDataset(train_df["text"].tolist(), train_df["label"].tolist(), tokenizer)
dev_dataset = TextDataset(dev_df["text"].tolist(), dev_df["label"].tolist(), tokenizer)
test_dataset = TextDataset(test_df["text"].tolist(), test_df["label"].tolist(), tokenizer)

# Create DataLoaders
batch_size = 32
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
dev_loader = DataLoader(dev_dataset, batch_size=batch_size, shuffle=False)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

# **Step 4: Capsule Network Layer**
class CapsuleLayer(nn.Module):
    def __init__(self, num_capsules, input_dim, output_dim, routing_iterations=3):
        super(CapsuleLayer, self).__init__()
        self.num_capsules = num_capsules
        self.input_dim = input_dim
        self.output_dim = output_dim
        self.routing_iterations = routing_iterations
        self.weights = nn.Parameter(torch.randn(num_capsules, input_dim, output_dim))

    def forward(self, x):
        x = x.unsqueeze(1).expand(-1, self.num_capsules, -1, -1)
        u_hat = torch.matmul(x, self.weights)

        b_ij = torch.zeros(u_hat.shape[0], self.num_capsules, u_hat.shape[2]).to(x.device)

        for _ in range(self.routing_iterations):
            c_ij = torch.nn.functional.softmax(b_ij, dim=1)
            s_j = (c_ij.unsqueeze(-1) * u_hat).sum(dim=2)
            v_j = self.squash(s_j)
            b_ij = b_ij + (u_hat * v_j.unsqueeze(2)).sum(dim=-1)

        return v_j

    @staticmethod
    def squash(tensor):
        norm = (tensor ** 2).sum(dim=-1, keepdim=True)
        scale = norm / (1 + norm)
        return scale * tensor / torch.sqrt(norm + 1e-8)

# **Step 5: Define CAPTAIN Model**
class CAPTAINModel(nn.Module):
    def __init__(self, num_classes=3):
        super(CAPTAINModel, self).__init__()
        self.bert = AutoModel.from_pretrained("csebuetnlp/banglabert")
        self.capsule = CapsuleLayer(num_capsules=8, input_dim=768, output_dim=16)
        self.flatten = nn.Flatten()
        self.fc1 = nn.Linear(8 * 16, 128)
        self.fc2 = nn.Linear(128, num_classes)
        self.batch_norm = nn.BatchNorm1d(128)
        self.dropout = nn.Dropout(0.5)

    def forward(self, input_ids, attention_mask):
        bert_output = self.bert(input_ids=input_ids, attention_mask=attention_mask).last_hidden_state
        caps_output = self.capsule(bert_output)
        flat_output = self.flatten(caps_output)
        x = self.batch_norm(torch.relu(self.fc1(flat_output)))
        x = self.dropout(x)
        logits = self.fc2(x)
        return torch.nn.functional.log_softmax(logits, dim=1)

# **Step 6: Training Setup**
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = CAPTAINModel().to(device)

optimizer = optim.AdamW(model.parameters(), lr=3e-5, weight_decay=1e-4)
criterion = nn.CrossEntropyLoss()
scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=5, gamma=0.7)

# **Step 7: Training Loop**
epochs = 50
for epoch in range(epochs):
    model.train()
    total_loss, correct, total = 0, 0, 0

    for batch in tqdm(train_loader, desc=f"Epoch {epoch+1}/{epochs}", leave=False):
        input_ids, attention_mask, labels = batch["input_ids"].to(device), batch["attention_mask"].to(device), batch["label"].to(device)

        optimizer.zero_grad()
        outputs = model(input_ids, attention_mask)

        loss = criterion(outputs, labels)
        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)
        optimizer.step()

        total_loss += loss.item()
        correct += (outputs.argmax(1) == labels).sum().item()
        total += labels.size(0)

    scheduler.step()
    print(f"Epoch {epoch+1} Loss: {total_loss:.4f}, Accuracy: {correct/total:.4f}")

# **Step 8: Model Evaluation**
model.eval()
predictions, true_labels = [], []
with torch.no_grad():
    for batch in test_loader:
        input_ids, attention_mask, labels = batch["input_ids"].to(device), batch["attention_mask"].to(device), batch["label"].to(device)

        outputs = model(input_ids, attention_mask)
        preds = torch.argmax(outputs, dim=1).cpu().numpy()
        labels = labels.cpu().numpy()

        predictions.extend(preds)
        true_labels.extend(labels)

# **Print Classification Report with 4-digit Precision**
print("Classification Report:")
print(classification_report(true_labels, predictions, digits=4))


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/119 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/586 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/528k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

...

Epoch 46 Loss: 0.4327, Accuracy: 0.9995


Epoch 47 Loss: 0.4400, Accuracy: 0.9998


Epoch 48 Loss: 0.4595, Accuracy: 1.0000


Epoch 49 Loss: 0.5882, Accuracy: 0.9995


Epoch 50 Loss: 0.5629, Accuracy: 0.9995
Classification Report:
              precision    recall  f1-score   support

           0     0.6968    0.8805    0.7779      1096
           1     0.8187    0.6223    0.7071      1096
           2     0.8346    0.8148    0.8246      1096

    accuracy                         0.7725      3288
   macro avg     0.7834    0.7725    0.7699      3288
weighted avg     0.7834    0.7725    0.7699      3288



# banglabert with stopwords

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
import pandas as pd
import numpy as np
import re
from torch.utils.data import Dataset, DataLoader
from transformers import AutoTokenizer, AutoModel
from tqdm import tqdm
from sklearn.metrics import accuracy_score, classification_report
from imblearn.over_sampling import RandomOverSampler  # Import RandomOverSampler
import unicodedata

# Load BanglaBERT Tokenizer
tokenizer = AutoTokenizer.from_pretrained("csebuetnlp/banglabert")

# Load Dataset
def load_data(file_path):
    df = pd.read_csv(file_path)
    df = df[['text', 'label']]
    return df

train_df = pd.read_csv("train_modified.csv")
dev_df = pd.read_csv("dev_modified.csv")
test_df = pd.read_csv("test_modified.csv")

# Step 1: Apply Random Oversampling to Balance the Datasets
def oversample_dataset(df):
    # Drop rows where 'text' or 'label' is NaN
    df = df.dropna(subset=['text', 'label'])

    # Ensure labels are integers
    df["label"] = df["label"].astype(int)

    ros = RandomOverSampler(random_state=42)
    texts, labels = df["text"].values.reshape(-1, 1), df["label"].values

    texts_resampled, labels_resampled = ros.fit_resample(texts, labels)

    balanced_df = pd.DataFrame({"text": texts_resampled.flatten(), "label": labels_resampled})
    return balanced_df

# Apply oversampling after fixing NaN issues
train_df = oversample_dataset(train_df)
dev_df = oversample_dataset(dev_df)
test_df = oversample_dataset(test_df)



# **Step 2: Preprocessing Function**
def preprocess_text(text):
    text = unicodedata.normalize('NFKC', text)  # Normalize Unicode text
    text = re.sub(r'[০-৯]', '[NUMBER]', text)  # Replace Bangla numbers
    text = re.sub(r'http\S+|www\S+', '[URL]', text)  # Replace URLs
    text = re.sub(r'[^\u0980-\u09FF\s]', '', text)  # Remove non-Bangla characters
    text = re.sub(r'\s+', ' ', text).strip()  # Normalize whitespace
    return text

train_df["text"] = train_df["text"].apply(preprocess_text)
dev_df["text"] = dev_df["text"].apply(preprocess_text)
test_df["text"] = test_df["text"].apply(preprocess_text)

# **Step 3: Define Dataset Class**
class TextDataset(Dataset):
    def __init__(self, texts, labels, tokenizer, max_length=70):
        self.texts = texts
        self.labels = labels
        self.tokenizer = tokenizer
        self.max_length = max_length

    def __len__(self):
        return len(self.texts)

    def __getitem__(self, idx):
        text = self.texts[idx]
        label = self.labels[idx]

        encoding = self.tokenizer(
            text,
            padding="max_length",
            truncation=True,
            max_length=self.max_length,
            return_tensors="pt"
        )

        return {
            "input_ids": encoding["input_ids"].squeeze(0),
            "attention_mask": encoding["attention_mask"].squeeze(0),
            "label": torch.tensor(label, dtype=torch.long)
        }

# Prepare datasets
train_dataset = TextDataset(train_df["text"].tolist(), train_df["label"].tolist(), tokenizer)
dev_dataset = TextDataset(dev_df["text"].tolist(), dev_df["label"].tolist(), tokenizer)
test_dataset = TextDataset(test_df["text"].tolist(), test_df["label"].tolist(), tokenizer)

# Create DataLoaders
batch_size = 32
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
dev_loader = DataLoader(dev_dataset, batch_size=batch_size, shuffle=False)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

# **Step 4: Capsule Network Layer**
class CapsuleLayer(nn.Module):
    def __init__(self, num_capsules, input_dim, output_dim, routing_iterations=3):
        super(CapsuleLayer, self).__init__()
        self.num_capsules = num_capsules
        self.input_dim = input_dim
        self.output_dim = output_dim
        self.routing_iterations = routing_iterations
        self.weights = nn.Parameter(torch.randn(num_capsules, input_dim, output_dim))

    def forward(self, x):
        x = x.unsqueeze(1).expand(-1, self.num_capsules, -1, -1)
        u_hat = torch.matmul(x, self.weights)

        b_ij = torch.zeros(u_hat.shape[0], self.num_capsules, u_hat.shape[2]).to(x.device)

        for _ in range(self.routing_iterations):
            c_ij = torch.nn.functional.softmax(b_ij, dim=1)
            s_j = (c_ij.unsqueeze(-1) * u_hat).sum(dim=2)
            v_j = self.squash(s_j)
            b_ij = b_ij + (u_hat * v_j.unsqueeze(2)).sum(dim=-1)

        return v_j

    @staticmethod
    def squash(tensor):
        norm = (tensor ** 2).sum(dim=-1, keepdim=True)
        scale = norm / (1 + norm)
        return scale * tensor / torch.sqrt(norm + 1e-8)

# **Step 5: Define CAPTAIN Model**
class CAPTAINModel(nn.Module):
    def __init__(self, num_classes=3):
        super(CAPTAINModel, self).__init__()
        self.bert = AutoModel.from_pretrained("csebuetnlp/banglabert")
        self.capsule = CapsuleLayer(num_capsules=8, input_dim=768, output_dim=16)
        self.flatten = nn.Flatten()
        self.fc1 = nn.Linear(8 * 16, 128)
        self.fc2 = nn.Linear(128, num_classes)
        self.batch_norm = nn.BatchNorm1d(128)
        self.dropout = nn.Dropout(0.8)

    def forward(self, input_ids, attention_mask):
        bert_output = self.bert(input_ids=input_ids, attention_mask=attention_mask).last_hidden_state
        caps_output = self.capsule(bert_output)
        flat_output = self.flatten(caps_output)
        x = self.batch_norm(torch.relu(self.fc1(flat_output)))
        x = self.dropout(x)
        logits = self.fc2(x)
        return torch.nn.functional.log_softmax(logits, dim=1)

# **Step 6: Training Setup**
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = CAPTAINModel().to(device)

optimizer = optim.AdamW(model.parameters(), lr=3e-5, weight_decay=1e-4)
criterion = nn.CrossEntropyLoss()
scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=5, gamma=0.7)

# **Step 7: Training Loop**
epochs = 30
for epoch in range(epochs):
    model.train()
    total_loss, correct, total = 0, 0, 0

    for batch in tqdm(train_loader, desc=f"Epoch {epoch+1}/{epochs}", leave=False):
        input_ids, attention_mask, labels = batch["input_ids"].to(device), batch["attention_mask"].to(device), batch["label"].to(device)

        optimizer.zero_grad()
        outputs = model(input_ids, attention_mask)

        loss = criterion(outputs, labels)
        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)
        optimizer.step()

        total_loss += loss.item()
        correct += (outputs.argmax(1) == labels).sum().item()
        total += labels.size(0)

    scheduler.step()
    print(f"Epoch {epoch+1} Loss: {total_loss:.4f}, Accuracy: {correct/total:.4f}")

# **Step 8: Model Evaluation**
model.eval()
predictions, true_labels = [], []
with torch.no_grad():
    for batch in test_loader:
        input_ids, attention_mask, labels = batch["input_ids"].to(device), batch["attention_mask"].to(device), batch["label"].to(device)

        outputs = model(input_ids, attention_mask)
        preds = torch.argmax(outputs, dim=1).cpu().numpy()
        labels = labels.cpu().numpy()

        predictions.extend(preds)
        true_labels.extend(labels)

# **Print Classification Report with 4-digit Precision**
print("Classification Report:")
print(classification_report(true_labels, predictions, digits=4))


#banglabert without stopwords

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
import pandas as pd
import numpy as np
import re
from torch.utils.data import Dataset, DataLoader
from transformers import AutoTokenizer, AutoModel
from tqdm import tqdm
from sklearn.metrics import accuracy_score, classification_report
from imblearn.over_sampling import RandomOverSampler  # Import RandomOverSampler
import unicodedata

# Load BanglaBERT Tokenizer
tokenizer = AutoTokenizer.from_pretrained("csebuetnlp/banglabert")

# Load Dataset
def load_data(file_path):
    df = pd.read_csv(file_path)
    df = df[['text', 'label']]
    return df

train_df = pd.read_csv("train_modified.csv")
dev_df = pd.read_csv("dev_modified.csv")
test_df = pd.read_csv("test_modified.csv")

# Load Stopwords
def load_stopwords(file_path="stopwords_bangla.csv"):
    stopwords_df = pd.read_csv(file_path)
    stopwords_list = set(stopwords_df["words"].dropna().tolist())  # Ensure no NaN values
    return stopwords_list

STOPWORDS = load_stopwords()  # Load stopwords once

# Step 1: Apply Random Oversampling to Balance the Datasets
def oversample_dataset(df):
    # Drop rows where 'text' or 'label' is NaN
    df = df.dropna(subset=['text', 'label'])

    # Ensure labels are integers
    df["label"] = df["label"].astype(int)

    ros = RandomOverSampler(random_state=42)
    texts, labels = df["text"].values.reshape(-1, 1), df["label"].values

    texts_resampled, labels_resampled = ros.fit_resample(texts, labels)

    balanced_df = pd.DataFrame({"text": texts_resampled.flatten(), "label": labels_resampled})
    return balanced_df

# Step 2: Preprocessing Function (Now includes Stopword Removal)
def preprocess_text(text):
    text = unicodedata.normalize('NFKC', text)  # Normalize Unicode text
    text = re.sub(r'[০-৯]', '[NUMBER]', text)  # Replace Bangla numbers
    text = re.sub(r'http\S+|www\S+', '[URL]', text)  # Replace URLs
    text = re.sub(r'[^\u0980-\u09FF\s]', '', text)  # Remove non-Bangla characters

    # Remove stopwords
    words = text.split()
    words = [word for word in words if word not in STOPWORDS]
    text = ' '.join(words)

    text = re.sub(r'\s+', ' ', text).strip()  # Normalize whitespace
    return text

# Apply preprocessing and oversampling
train_df = oversample_dataset(train_df)
dev_df = oversample_dataset(dev_df)
test_df = oversample_dataset(test_df)

train_df["text"] = train_df["text"].apply(preprocess_text)
dev_df["text"] = dev_df["text"].apply(preprocess_text)
test_df["text"] = test_df["text"].apply(preprocess_text)


# **Step 3: Define Dataset Class**
class TextDataset(Dataset):
    def __init__(self, texts, labels, tokenizer, max_length=70):
        self.texts = texts
        self.labels = labels
        self.tokenizer = tokenizer
        self.max_length = max_length

    def __len__(self):
        return len(self.texts)

    def __getitem__(self, idx):
        text = self.texts[idx]
        label = self.labels[idx]

        encoding = self.tokenizer(
            text,
            padding="max_length",
            truncation=True,
            max_length=self.max_length,
            return_tensors="pt"
        )

        return {
            "input_ids": encoding["input_ids"].squeeze(0),
            "attention_mask": encoding["attention_mask"].squeeze(0),
            "label": torch.tensor(label, dtype=torch.long)
        }

# Prepare datasets
train_dataset = TextDataset(train_df["text"].tolist(), train_df["label"].tolist(), tokenizer)
dev_dataset = TextDataset(dev_df["text"].tolist(), dev_df["label"].tolist(), tokenizer)
test_dataset = TextDataset(test_df["text"].tolist(), test_df["label"].tolist(), tokenizer)

# Create DataLoaders
batch_size = 32
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
dev_loader = DataLoader(dev_dataset, batch_size=batch_size, shuffle=False)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

# **Step 4: Capsule Network Layer**
class CapsuleLayer(nn.Module):
    def __init__(self, num_capsules, input_dim, output_dim, routing_iterations=3):
        super(CapsuleLayer, self).__init__()
        self.num_capsules = num_capsules
        self.input_dim = input_dim
        self.output_dim = output_dim
        self.routing_iterations = routing_iterations
        self.weights = nn.Parameter(torch.randn(num_capsules, input_dim, output_dim))

    def forward(self, x):
        x = x.unsqueeze(1).expand(-1, self.num_capsules, -1, -1)
        u_hat = torch.matmul(x, self.weights)

        b_ij = torch.zeros(u_hat.shape[0], self.num_capsules, u_hat.shape[2]).to(x.device)

        for _ in range(self.routing_iterations):
            c_ij = torch.nn.functional.softmax(b_ij, dim=1)
            s_j = (c_ij.unsqueeze(-1) * u_hat).sum(dim=2)
            v_j = self.squash(s_j)
            b_ij = b_ij + (u_hat * v_j.unsqueeze(2)).sum(dim=-1)

        return v_j

    @staticmethod
    def squash(tensor):
        norm = (tensor ** 2).sum(dim=-1, keepdim=True)
        scale = norm / (1 + norm)
        return scale * tensor / torch.sqrt(norm + 1e-8)

# **Step 5: Define CAPTAIN Model**
class CAPTAINModel(nn.Module):
    def __init__(self, num_classes=3):
        super(CAPTAINModel, self).__init__()
        self.bert = AutoModel.from_pretrained("csebuetnlp/banglabert")
        self.capsule = CapsuleLayer(num_capsules=8, input_dim=768, output_dim=16)
        self.flatten = nn.Flatten()
        self.fc1 = nn.Linear(8 * 16, 128)
        self.fc2 = nn.Linear(128, num_classes)
        self.batch_norm = nn.BatchNorm1d(128)
        self.dropout = nn.Dropout(0.5)

    def forward(self, input_ids, attention_mask):
        bert_output = self.bert(input_ids=input_ids, attention_mask=attention_mask).last_hidden_state
        caps_output = self.capsule(bert_output)
        flat_output = self.flatten(caps_output)
        x = self.batch_norm(torch.relu(self.fc1(flat_output)))
        x = self.dropout(x)
        logits = self.fc2(x)
        return torch.nn.functional.log_softmax(logits, dim=1)

# **Step 6: Training Setup**
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = CAPTAINModel().to(device)

optimizer = optim.AdamW(model.parameters(), lr=3e-5, weight_decay=1e-4)
criterion = nn.CrossEntropyLoss()
scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=5, gamma=0.7)

# **Step 7: Training Loop**
epochs = 30
for epoch in range(epochs):
    model.train()
    total_loss, correct, total = 0, 0, 0

    for batch in tqdm(train_loader, desc=f"Epoch {epoch+1}/{epochs}", leave=False):
        input_ids, attention_mask, labels = batch["input_ids"].to(device), batch["attention_mask"].to(device), batch["label"].to(device)

        optimizer.zero_grad()
        outputs = model(input_ids, attention_mask)

        loss = criterion(outputs, labels)
        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)
        optimizer.step()

        total_loss += loss.item()
        correct += (outputs.argmax(1) == labels).sum().item()
        total += labels.size(0)

    scheduler.step()
    print(f"Epoch {epoch+1} Loss: {total_loss:.4f}, Accuracy: {correct/total:.4f}")

# **Step 8: Model Evaluation**
model.eval()
predictions, true_labels = [], []
with torch.no_grad():
    for batch in test_loader:
        input_ids, attention_mask, labels = batch["input_ids"].to(device), batch["attention_mask"].to(device), batch["label"].to(device)

        outputs = model(input_ids, attention_mask)
        preds = torch.argmax(outputs, dim=1).cpu().numpy()
        labels = labels.cpu().numpy()

        predictions.extend(preds)
        true_labels.extend(labels)

# **Print Classification Report with 4-digit Precision**
print("Classification Report:")
print(classification_report(true_labels, predictions, digits=4))


In [ ]:
!pip install --upgrade pip
!pip install --upgrade tensorflow torch transformers
!pip install tqdm



In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
import pandas as pd
import numpy as np
import re
from torch.utils.data import Dataset, DataLoader
from transformers import AutoTokenizer, AutoModel
from tqdm import tqdm
from sklearn.metrics import accuracy_score, classification_report
from imblearn.over_sampling import RandomOverSampler
import unicodedata

# Load XLM-RoBERTa Tokenizer
tokenizer = AutoTokenizer.from_pretrained("xlm-roberta-large")

# Load Dataset
def load_data(file_path):
    df = pd.read_csv(file_path)
    df = df[['text', 'label']]
    return df.dropna(subset=['text', 'label'])  # Drop missing values

train_df = load_data("train_modified.csv")
dev_df = load_data("dev_modified.csv")
test_df = load_data("test_modified.csv")

# Apply Oversampling
def oversample_dataset(df):
    df["label"] = df["label"].astype(int)  # Ensure labels are integers
    ros = RandomOverSampler(random_state=42)

    texts, labels = df["text"].values.reshape(-1, 1), df["label"].values
    texts_resampled, labels_resampled = ros.fit_resample(texts, labels)

    return pd.DataFrame({"text": texts_resampled.flatten(), "label": labels_resampled})

train_df = oversample_dataset(train_df)  # Oversample only training data

# Preprocessing Function
def preprocess_text(text):
    text = unicodedata.normalize('NFKC', text)
    text = re.sub(r'[০-৯]', '[NUMBER]', text)  # Replace Bengali numbers
    text = re.sub(r'http\S+|www\S+', '[URL]', text)  # Replace URLs
    text = re.sub(r'[^\u0980-\u09FF\s]', '', text)  # Keep only Bengali text
    text = re.sub(r'\s+', ' ', text).strip()
    return text

# Apply Preprocessing
for df in [train_df, dev_df, test_df]:
    df["text"] = df["text"].apply(preprocess_text)

# Dataset Class
class TextDataset(Dataset):
    def __init__(self, texts, labels, tokenizer, max_length=70):
        self.texts = texts
        self.labels = labels
        self.tokenizer = tokenizer
        self.max_length = max_length

    def __len__(self):
        return len(self.texts)

    def __getitem__(self, idx):
        text = str(self.texts[idx])  # Ensure it's a string
        label = self.labels[idx]

        encoding = self.tokenizer(
            text,
            padding="max_length",
            truncation=True,
            max_length=self.max_length,
            return_tensors="pt"
        )

        return {
            "input_ids": encoding["input_ids"].squeeze(),  # Removed squeeze(0) to avoid shape issues
            "attention_mask": encoding["attention_mask"].squeeze(),
            "label": torch.tensor(label, dtype=torch.long)
        }

# Prepare datasets and DataLoaders
batch_size = 32
train_dataset = TextDataset(train_df["text"].tolist(), train_df["label"].tolist(), tokenizer)
dev_dataset = TextDataset(dev_df["text"].tolist(), dev_df["label"].tolist(), tokenizer)
test_dataset = TextDataset(test_df["text"].tolist(), test_df["label"].tolist(), tokenizer)

train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
dev_loader = DataLoader(dev_dataset, batch_size=batch_size, shuffle=False)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

# Capsule Network Layer
class CapsuleLayer(nn.Module):
    def __init__(self, num_capsules, input_dim, output_dim, routing_iterations=3):
        super(CapsuleLayer, self).__init__()
        self.num_capsules = num_capsules
        self.input_dim = input_dim
        self.output_dim = output_dim
        self.routing_iterations = routing_iterations
        self.weights = nn.Parameter(torch.randn(num_capsules, input_dim, output_dim))

    def forward(self, x):
        x = x.unsqueeze(1).expand(-1, self.num_capsules, -1, -1)
        u_hat = torch.matmul(x, self.weights)

        b_ij = torch.zeros(u_hat.shape[0], self.num_capsules, u_hat.shape[2]).to(x.device)

        for _ in range(self.routing_iterations):
            c_ij = torch.nn.functional.softmax(b_ij, dim=1)
            s_j = (c_ij.unsqueeze(-1) * u_hat).sum(dim=2)
            v_j = self.squash(s_j)
            b_ij = b_ij + (u_hat * v_j.unsqueeze(2)).sum(dim=-1)

        return v_j

    @staticmethod
    def squash(tensor):
        norm = (tensor ** 2).sum(dim=-1, keepdim=True)
        scale = norm / (1 + norm)
        return scale * tensor / torch.sqrt(norm + 1e-8)

# Define CAPTAIN Model
class CAPTAINModel(nn.Module):
    def __init__(self, num_classes=3):
        super(CAPTAINModel, self).__init__()
        self.bert = AutoModel.from_pretrained("xlm-roberta-large")
        self.capsule = CapsuleLayer(num_capsules=8, input_dim=768, output_dim=16)
        self.flatten = nn.Flatten()
        self.fc1 = nn.Linear(8 * 16, 128)
        self.fc2 = nn.Linear(128, num_classes)
        self.batch_norm = nn.BatchNorm1d(128)
        self.dropout = nn.Dropout(0.5)

    def forward(self, input_ids, attention_mask):
        bert_output = self.bert(input_ids=input_ids, attention_mask=attention_mask).last_hidden_state
        caps_output = self.capsule(bert_output)
        flat_output = self.flatten(caps_output)
        x = self.batch_norm(torch.relu(self.fc1(flat_output)))
        x = self.dropout(x)
        logits = self.fc2(x)
        return torch.nn.functional.log_softmax(logits, dim=1)

# Training Setup
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = CAPTAINModel().to(device)

optimizer = optim.AdamW(model.parameters(), lr=3e-5, weight_decay=1e-4)
criterion = nn.CrossEntropyLoss()
scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=5, gamma=0.7)

# Training Loop
epochs = 30
for epoch in range(epochs):
    model.train()
    total_loss, correct, total = 0, 0, 0

    for batch in tqdm(train_loader, desc=f"Epoch {epoch+1}/{epochs}", leave=False):
        input_ids, attention_mask, labels = batch["input_ids"].to(device), batch["attention_mask"].to(device), batch["label"].to(device)

        optimizer.zero_grad()
        outputs = model(input_ids, attention_mask)

        loss = criterion(outputs, labels)
        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)
        optimizer.step()

        total_loss += loss.item()
        correct += (outputs.argmax(1) == labels).sum().item()
        total += labels.size(0)

    scheduler.step()
    print(f"Epoch {epoch+1} Loss: {total_loss:.4f}, Accuracy: {correct/total:.4f}")

# Model Evaluation
model.eval()
predictions, true_labels = [], []
with torch.no_grad():
    for batch in test_loader:
        input_ids, attention_mask, labels = batch["input_ids"].to(device), batch["attention_mask"].to(device), batch["label"].to(device)

        outputs = model(input_ids, attention_mask)
        preds = torch.argmax(outputs, dim=1).cpu().numpy()
        true_labels.extend(labels.cpu().numpy())
        predictions.extend(preds)

print("Classification Report:")
print(classification_report(true_labels, predictions, digits=4))
